# 3070 running hot

In [ ]:
print("Hello world")

In [1]:
#Check if cuda is available and running on the right gpu
import torch

torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3070'

In [ ]:
!python test_data_balanced.py

In [ ]:
!python train_data.py

Preprocess the images for training, so that the Image Transformation does not have to be done for each training attempt.
The Transformation is pretty CPU and Disk I/O heavy.
Loading the preprocessed tensors eliminates that load. Also the tensors are kept in VRAM for training.

In [ ]:
#%run preprocess_transform.py --data_dir ./inputs/training --output_dir ./inputs/transformed_training
#%run preprocess_transform.py --data_dir ./inputs/test_balanced --output_dir ./inputs/transformed_test_balanced

%run preprocess_transform.py --dataset train
%run preprocess_transform.py --dataset test_balanced

good batch_sizes would be denominators of 4800
100 92% GPU, 2.1G VRAM, 5s
200 90% GPU, 2.4G VRAM, 5s
300 93% GPU, 3.1G VRAM, 4s
400 93% GPU, 3.4G VRAM, 4s
600 93% GPU, 4.7G VRAM, 4s
800 93% GPU, 6.0G VRAM, 4s
1200 93% GPU, 7.7G VRAM, 5s

In [ ]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

main_train(
    model_name='mobilenet_v2',
    data_dir_train='inputs/data_train_local',        # Before was "transformed_training" 
    data_dir_test='inputs/data_test_local',
    output_model_dir='outputs',
    batch_size=200,
    num_epochs=1,
    learning_rate=0.0035148759,
    resume=False,
    test=True
)

# %run train.py \
#     --data_dir_train inputs/transformed_training \
#     --batch_size 200 \
#     --num_epoch 10 \
#     --test True \
#     --data_dir_test ./inputs/transformed_test_balanced \
#     --output_model_dir ./outputs

Run this in Terminal

```sh
uv run train.py
    --data_dir_train inputs/transformed_training
    --batch_size 200
    --num_epoch 3000
    --test True
    --data_dir_test inputs/transformed_test_balanced
    --output_model_dir outputs
```

`uv run tensorboard --logdir=runs`

# Running Vision Transformer

In [3]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

learning_rates = [0.01, 0.005, 0.001, 0.0001]

for lr in learning_rates: 
    main_train(
        model_name='vit_base_patch16_224',
        data_dir_train='inputs/data_train_local',
        data_dir_test='inputs/data_test_local',
        output_model_dir='outputs_vit',
        num_epochs = 600, 
        learning_rate=lr,
        test_every_x_epochs=3,
        resume=False,
        test=True, 
        metric=True
    )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cuda
Model: vit_base_patch16_224
Hyperparameters: {'learning_rate': 0.01, 'batch_size': 1000, 'num_epochs': 600, 'test_every_x_epochs': 3}
Train Dataset: Found 4800 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Test Dataset: Found 1020 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']


c:\Users\Farmer\Desktop\FarmersEye\farmers-eye-pytorch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Moving model to cuda
Compiling model

Starting training from epoch 1 to 600
Logging to TensorBoard directory: ./runs\vit_base_patch16_224_20250625_020155
Run 'tensorboard --logdir=runs' to view training progress



Epoch 1/600: 5it [00:17,  3.60s/it]                       


Epoch 1/600 - Train Loss: 2.5857, Train Acc: 0.1671


...

Epoch 599/600 - Train Loss: 0.7603, Train Acc: 0.7667


Epoch 600/600: 5it [00:08,  1.69s/it]                       


Epoch 600/600 - Train Loss: 0.7598, Train Acc: 0.7665


Evaluating: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


Test Loss: 0.7720, Test Acc: 0.7598 (Best: 0.7598)
Model saved to outputs_vit\000600_model.pth

Training completed!
TensorBoard logs saved to: ./runs\vit_base_patch16_224_20250625_063842
View with: tensorboard --logdir=./runs


In [ ]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

main_train(
    model_name='vit_huge_patch14_224',
    data_dir_train='inputs/transformed_training',
    data_dir_test='inputs/transformed_test_balanced',
    output_model_dir='outputs_vit',
    resume=False,
    test=True
)

```sh

# Foundation Model